**<span style="color:OrangeRed; font-size:250%">Nighteen Subject-specific statistics modeling using STATA </span>**

>
> <span style="color:moccasin">By </span> <span style="color:orange">M. **Ibrahima TALL** </span>
>
> <span style="color:orchid"> Statistician at ANSD (Statistic and Demography National agency)</span>
>

****

<span style="color:peru">Bayesian analysis starts with</span> <span style="color:gold"> the specification of a **posterior** model.</span> <span style="color:peru"> The **posterior** distribution has two components: a likelihood, which includes information about model parameters based on the bserved data, and a prior, which includes prior  information (before observing the data) about model parameters. **<span style="color:gold">Inference</span>** <span style="color:peru">  is the next step of Bayesian analysis. If MCMC sampling is used for approximating the posterior distribution, the convergence of MCMC (Markov Chain Monte Carlo) must be established before proceeding to inference.</span> <span style="color:peru">Another important step of Bayesian analysis is</span> **<span style="color:gold">model checking</span>** <span style="color:peru">, which is typically performed via posterior predictive checking.</span> **<span style="color:gold">Model comparison</span>** <span style="color:peru">is another common step of Bayesian analysis. Finally,</span> **<span style="color:gold">prediction</span>** <span style="color:peru">of some future unobserved data may also be of interest in Bayesian analysis.</span>

**<span style="color:crimson;font-size:150%"> Table of contents </span>**  

1. <span style="color:olive">Bayesian analysis</span>  
2. <span style="color:olive">Choice Models</span>
3. <span style="color:olive">Dynamic Stochastic General Equilibrium Models</span>
4. <span style="color:olive">Extended Regression Models</span>
5. <span style="color:olive">Finite Mixture Models</span>
6. <span style="color:olive">Item Response Theory</span>
7. <span style="color:olive">Lasso</span>
8. <span style="color:olive">Longitudinal-Data/Panel-Data</span>
9. <span style="color:olive">Meta-Analysis</span>
10. <span style="color:olive">Multilevel Mixed-Effects</span>
11. <span style="color:olive">Multiple-Imputation</span>
12. <span style="color:olive">Multivariate Statistics</span>
13. <span style="color:olive">Power, Precision, and Sample-Size</span>
14. <span style="color:olive">Spatial Autoregressive Models</span>
15. <span style="color:olive">Structural Equation Modeling</span>
16. <span style="color:olive">Survey Data</span>
17. <span style="color:olive">Survival Analysis</span>
18. <span style="color:olive">Time Series</span>
19. <span style="color:olive">Treatment Effects </span>

<span style="color:peru">In each chapter, an real examples will studed to show all STATA command syntax using in coresponding cases.</span>  

# **<span style="color:crimson;font-size:150%"> I. Bayesian analysis </span>**

In [ ]:
use oxygen, clear
describe
regress change group age //OLS

In [ ]:
* Bayesian normal linear regression with noninformative prior
bayesmh change group age, likelihood(normal({var})) prior({change:}, flat) prior({var}, jeffreys)
/* The lower bound of the interval is greater than 0, so we conclude that there is an effect of the exercise program 
on the change in oxygen uptake. A low acceptance rate (for example, below 10%) may indicate convergence problems*/

In [ ]:
*Bayesian linear regression with informative prior
set seed 14
bayesmh change group age, likelihood(normal({var})) prior({change:}, normal(0, {var})) prior({var}, igamma(2.5, 2.5))

In [ ]:
/* Bayesian normal linear regression with multivariate prior: Zellner’s g-prior the more commonly 
   used priors for the regression coefficients in a normal linear regression */
set seed 14
bayesmh change group age, likelihood(normal({var})) prior({change:}, zellnersg0(3,12,{var})) prior({var}, igamma(0.5, 4))

In [ ]:
* We can reproduce what zellnersg0() does above manually
matrix accum xTx = group age
matrix accum xTx = group age
matrix S = syminv(xTx)
set seed 14
bayesmh change group age, likelihood(normal({var})) prior({change:}, mvnormal0(3,12*{var}*S)) prior({var}, igamma(0.5, 4))

In [ ]:
* Checking convergence
bayesgraph diagnostics {change:group}
bayesgraph diagnostics _all //To see all model
bayesstats ess // Effective sample sizes and statistics related to them
bayesgraph diagnostics{var} 
* The closer ESS estimates are to the MCMC sample size, the less correlated the MCMC 
* sample is, and the more precise our estimates of parameters are*/
* Putting the variance parameter in a separate block
bayesmh change group age, likelihood(normal({var})) prior({change:}, zellnersg0(3,12,{var})) ///
 prior({var}, igamma(0.5, 4)) block({var}) saving(agegroup_simdata)
estimates store agegroup //store current estimation results as agegroup for future use
bayesgraph diagnostics {change:group}{var}

In [ ]:
* Postestimation summaries
summarize group
scalar sd_x = r(sd)
summarize change
scalar sd_y = r(sd)
bayesstats summary (group_std:{change:group}*sd_x/sd_y)

In [ ]:
* Bayesian predictions
bayespredict {_ysim}, saving(change_pred) rseed(16)
bayesstats summary {_ysim} using change_pred //calculate posterior summaries
bayesstats ppvalues {_ysim} using change_pred /*assess goodness of fit of the model is by comparing 
                                                replicated outcome samples with the observed outcome sample. 
                                                The discrepancy between these two can be measured using the
                                                so-called posterior predictive p-values */

In [ ]:
* We want to predict the outcome change for the new observations
set obs 14
replace group = 1 in 13
replace group = 0 in 14
replace age = 26 in 13/14
bayespredict pmean, mean rseed(16)
/* Finally, we drop the two new observations we added and erase the prediction dataset and the
auxiliary estimation file created by bayespredict */
drop in 13/14
erase change_pred.dta
erase change_pred.ster

In [ ]:
* Model comparison
set seed 14
bayesmh change group age ageXgr, likelihood(normal({var})) prior({change:}, zellnersg0(4,12,{var})) ///
    prior({var}, igamma(0.5, 4)) block({var}) saving(full_simdata)

estimates store full
bayesstats ic full agegroup //Kass and Raftery table scale, 2 × 1.28 = 2.56 is greater than 2: so agegroup < full

In [ ]:
* Hypothesis testing
bayestest model full agegroup

estimates restore agegroup //load the results of the agegroup model back into memory
bayestest interval {change:group}, lower(4) upper(8) //compute probability that coefficient for group between 4 and 8

In [ ]:
* Erasing simulation datasets: remove both simulation files we created using bayesmh
erase agegroup_simdata.dta
erase full_simdata.dta

In [ ]:
* Bayesian linear regression using the bayes prefix
set seed 14
bayes: regress change group age

In [ ]:
set seed 14
bayes, gibbs: regress change group age //Request Gibbs sampling. Available only with bayes: regress and bayes: mvreg

In [ ]:
set seed 14
bayes, prior({change:}, zellnersg0(3,12,{sigma2})) prior({sigma2}, igamma(0.5, 4)): regress change group age

In [ ]:
set seed 14
bayes, prior({change:}, zellnersg0(3,12,{sigma2})) prior({sigma2}, igamma(0.5, 4)) noblocking: regress change group age

# **<span style="color:crimson;font-size:150%"> II. Choice Model (Modelisation des choix)</span>**

**<span style="color:DarkGreen;font-size:200%"> II.1 Interpretation of choice models </span>**

In [ ]:
* Interpretation of coefficients
use https://www.stata-press.com/data/r16/travel, clear

In [ ]:
desc

In [ ]:
generate time = traveltime+termtime // total travel time

In [ ]:
/* We use cmset to specify that the id variable identifies the travelers and the mode variable records the possible
    methods of travel, called alternatives */
cmset id mode

In [ ]:
cmclogit choice time, casevars(income partysize) //casevars() option for only one value per individual (or case)

In [ ]:
margins // obtain the average predicted probability of choosing each method of travel

In [ ]:
margins, at(income=(30(10)70)) outcome(car) // Expected probabilities of car travel for counterfactual income
marginsplot // visualize the effect of income on car travel

In [ ]:
/* test for differences in the expected probabilities: nowald and effects options to simplify output */
margins, at(income=(30(10)70)) outcome(car) contrast(atcontrast(ar) nowald effects)

In [ ]:
margins, at(income=(30(10)70)) outcome(train) // Effet on train travel instead of car as above
marginsplot
margins, at(income=(30(10)70)) outcome(train) contrast(atcontrast(ar) nowald effects)

In [ ]:
margins, at(income=(30(10)70)) // for each method of travel
marginsplot, noci // omit the confidence intervals so that it is easy to see the probabilities
margins, at(income=30) contrast(outcomejoint) // Testing if all four methods of travel equally likely to be selected

In [ ]:
* difference in the expected probabilities of selecting train and bus travel
margins, at(income=50) outcome(bus train) contrast(outcomecontrast(r) nowald effects)

In [ ]:
* Instead of effect on continu variable (income) what is effect on categorical variable
xtile income_cat = income, nquantiles(4)
label define quartiles 1 "Quartile 1" 2 "Quartile 2" 3 "Quartile 3" 4 "Quartile 4"
label values income_cat quartiles

cmclogit choice time, casevars(i.income_cat partysize)
margins income_cat, outcome(train)
marginsplot

* We use the ar. operator to test for differences in the expected probabilities across adjacent income quartiles
margins ar.income_cat, outcome(train) contrast(nowald effects) //ar.income_cat because of categorical type

In [ ]:
* Wait times at the airport increase by 60 minutes for all flights?
margins, at(time=generate(time+60)) alternative(air)

In [ ]:
margins, at(time=generate(time)) at(time=generate(time+60)) alternative(air) // The two probabilities for comparaison

In [ ]:
* Plot two scenarios together include the xdimension() option to place the four travel choices along the x axis
marginsplot, xdimension(_outcome)

In [ ]:
* contrasts to test for a difference: atcontrast to specify test between at 
margins, at(time=generate(time)) at(time=generate(time+60)) alternative(air) contrast(atcontrast(r) nowald effects)

In [ ]:
* Testing new scenario
generate newtime = time
replace newtime = time + 60 if mode == 1
replace newtime = time-60 if mode == 2

* alternative(simultaneous) option to specify that the changes to air and train travel be made simultaneously
margins, at(time=generate(time)) at(time=generate(newtime)) alternative(simultaneous)
marginsplot, xdimension(_outcome)

* contrasts to test for a difference: atcontrast to specify test between at 
margins, at(time=generate(time)) at(time=generate(newtime)) alternative(simultaneous) contrast(atcontrast(r) nowald effects)

**<span style="color:DarkGreen;font-size:200%"> II.2 Data layout </span>**

In [ ]:
* Data layout for choice models: requiert long format instead of wide
use http://www.stata-press.com/data/r16/carchoice, clear

ist consumerid car purchase gender income dealers if consumerid <= 3, sepby(consumerid) abbrev(10)
/*
***************************************************************************************
    consumerid = case ID variable
    car = alternatives
    purchase = Choice (For discrete choice models dependent variable in the estimation)
    gender and income = case-specific variables; they are constant within case
    dealers = alternative-specific variable
***************************************************************************************
*/

In [ ]:
* cmset: Cross-sectional data
cmset consumerid car // we pass the case ID variable and alternatives variable as arguments
cmchoiceset // Tabulation of choice-set possibilities

In [ ]:
cmset consumerid, noalternatives // When there is no alternatives variable as in cmrologit estimator

In [ ]:
* cmset: Panel data
use http://www.stata-press.com/data/r16/transport.dta, clear
list id t alt if id == 1, sepby(t)

cmset id t alt // For panel choice data, cmset takes three variables

**<span style="color:DarkGreen;font-size:200%"> II.3 Descriptive statistics </span>**

In [ ]:
* Descriptive statistics using car choice data again
use http://www.stata-press.com/data/r16/carchoice, clear
list consumerid car purchase gender income if consumerid <= 3, sepby(consumerid) abbrev(10)
cmset consumerid car
cmchoiceset
label list nation
cmclogit purchase dealers, casevars(i.gender income) // cmchoiceset can be used after a cm estimation
cmchoiceset if e(sample) // cmchoiceset restricted to the estimation sample. missing values are handled casewise

cmclogit purchase dealers, casevars(i.gender income) altwise // altwise not suppress entire case, just line missing
cmchoiceset if e(sample) // Now total number of observations increase
cmtab, choice(purchase) //
cmtab gender, choice(purchase) column // cmtab with a variable gives a tabulation
cmsummarize income, choice(purchase) stats(p25 p50 p75) format(%5.1f) // Summarize stats


**<span style="color:DarkGreen;font-size:200%"> II.4 Models for discrete choices </span>**

<span style="color:peru">Discrete choices models are:</span>  
- <span style="color:olive">**cmclogit**: McFadden’s choice model</span>
- <span style="color:olive">**cmmixlogit**: Mixed logit choice models</span>
- <span style="color:olive">**cmmprobit**: Multinomial probit choice models</span>
- <span style="color:olive">**nlogit**: Nested logit choice models</span>

In [ ]:
******************************************************************************************************************
*************************************** cmclogit: McFadden’s choice model ****************************************
******************************************************************************************************************

/* Note that alternative-specific variables (if any) follow the dependent variable. Case-specific variables
(if any) are placed in the option casevars() */
cmclogit purchase dealers, casevars(i.gender income) // Conditional logit regression model

* Looking at cases with missing values using cmsample
cmsample dealers, choice(purchase) casevars(i.gender income) // Missing for model above: in the casevars
cmsample, casevars(i.gender) generate(flag) // Missing at gender alone with cmsample and gen flag to indicate missing

sort consumerid car // sorting and listing missing cases
list consumerid car gender flag if flag != 0, sepby(consumerid) abbr(10)

* margins after CM estimation
margins, at(dealers=generate(dealers+1)) alternative(European)
margins, at(dealers=generate(dealers)) at(dealers=generate(dealers+1)) alternative(European) contrast(atcontrast(r) nowald)
* Only those consumers who have Korean in their choice set, we can use the outcome( : : : , altsubpop) option
margins, at(dealers=generate(dealers)) at(dealers=generate(dealers+1)) alternative(European) contrast(atcontrast(r) nowald) outcome(Korean, altsubpop)

In [ ]:
******************************************************************************************************************
*************************************** cmmixlogit: Mixed logit choice models ************************************
******************************************************************************************************************

* Like cmclogit, cmmixlogit is used to model the probability that a decision maker chooses one alternative
* Mixed logit choice models can fit random coefficients for alternative-specific variables
cmmixlogit purchase, random(dealers) casevars(i.gender income)

In [ ]:
* We fit the model again using a lognormal distribution for the coefficient of dealers
cmmixlogit purchase, random(dealers, lognormal) casevars(i.gender income)

In [ ]:
/* Because we want
fixed coefficients on Japanese and Korean indicators, we type i(2 4).car in the fixed portion of the
model. To get random coefficients for the European constant, we type random(i3.car). We also
specify the options noconstant and collinear (or else cmmixlogit would drop the constants)*/
cmmixlogit purchase dealers i(2 4).car, random(i3.car) casevars(i.gender income) noconstant collinear

In [ ]:
******************************************************************************************************************
********************************* cmmprobit: Multinomial probit choice models ************************************
******************************************************************************************************************

* cmmprobit: Multinomial probit choice models
cmmprobit purchase dealers, casevars(i.gender income) factor(1)
estat covariance
estat correlation

cmmprobit purchase dealers, casevars(i.gender income) factor(1) structural
estat covariance
estat correlation

In [ ]:
******************************************************************************************************************
********************************* nlogit: Nested logit choice models *********************************************
******************************************************************************************************************

/* nlogit: Nested logit choice models : data could represent first-level choices of what restaurant to dine at 
and second-level choices of what is ordered at the restaurant.The second-level are conditional on the first-level
This is not a cm command
*/
help nlogit

**<span style="color:peru"> Relationships with other estimation commands</span>**

In [ ]:
* Duplicating cmclogit using clogit: for comparaison
drop if flag != 0 // Dealing with missing
clogit purchase dealers car##gender car##c.income, group(consumerid) // Same with cmclogit

In [ ]:
/* Multinomial logistic regression and McFadden’s choice model: Multinomial logistic regression (mlogit) is a 
    special case of McFadden’s choice model (cmclogit)*/
cmchoiceset, generate(choiceset) 
keep if choiceset == "1 2 3 4":choiceset // We will only work with balanced data

cmclogit purchase, casevars(i.gender income)
keep if purchase == 1 // To fit mlogit
mlogit car i.gender income // The estimates are identical

In [ ]:
use https://www.stata-press.com/data/r16/carchoice, clear
cmset consumerid car
quietly cmmprobit purchase dealers, casevars(i.gender income) factor(1) intpoints(2000) // Integration points up704
matrix b2000 = e(b)
quietly cmmprobit purchase dealers, casevars(i.gender income) factor(1) intpoints(4000) // Integration points up704
matrix b4000 = e(b)
display mreldif(b704, b2000) // Relative difference between matrices

**<span style="color:DarkGreen;font-size:200%">II.5 Models for rank-ordered alternatives </span>**

<span style="color:peru">Rank-ordered alternatives models are:</span>  
- <span style="color:olive">**cmroprobit**: Probit regression for rank-ordered alternatives</span>
- <span style="color:olive">**cmrologit**: Logistic regression for rank-ordered alternatives</span>

In [ ]:
******************************************************************************************************************
************************ cmroprobit: Probit regression for rank-ordered alternatives *****************************
******************************************************************************************************************

use http://www.stata-press.com/data/r16/wlsrank, clear
keep if noties
generate currentjob = 1 if low==1
replace currentjob = 2 if low==0 & high==0
replace currentjob = 3 if high==1
label define current 1 "Low" 2 "Neither" 3 "High"
label values currentjob current
list id jobchar rank female score currentjob in 1/12, sepby(id)

cmset id jobchar
* cmroprobit: Probit regression for rank-ordered alternatives
cmroprobit rank i.currentjob, casevars(i.female score) reverse structural
estat covariance
estat correlation
margins 3.currentjob, alternative(security)

In [ ]:
******************************************************************************************************************
************************ cmrologit: Logistic regression for rank-ordered alternatives ****************************
******************************************************************************************************************

list pref female age grades edufit workexp if caseid == 7, noobs
cmset caseid, noalternatives
cmrologit pref i.female i.age i.grades i.edufit i.workexp, baselevels

**<span style="color:DarkGreen;font-size:200%"> II.6 Models for panel data </span>**

In [ ]:
use http://www.stata-press.com/data/r16/transport //mixed logit model to panel choice data
list if id <= 2, sepby(t)
cmset id t alt
list id t alt _caseid _panelaltid if id <= 2, sepby(alt) abbr(11)
cmxtmixlogit choice trcost, random(trtime) casevars(age income)

xtset // Time-series operators
cmxtmixlogit choice, random(trtime L.trtime, correlated) casevars(age income)

* Using other cm estimation commands with panel data
cmclogit choice trcost trtime, casevars(age income)

# **<span style="color:crimson;font-size:150%"> III. Dynamic Stochastic General Equilibrum</span>**

**<span style="color:DarkGreen;font-size:200%"> III.1 Introduction to DSGEs </span>**

**<span style="color:DarkGreen;font-size:150%"> An example: A nonlinear DSGE model </span>**

In [ ]:
****************************************** Data preparation ********************************************************
use http://www.stata-press.com/data/r16/rates2, clear
tsset // First, the data must be tsset
describe
/* Model is written in terms of the inflation rate. For quarterly data, 
the inflation rate is conventionally obtained as 400 times the difference in log of the price level */
generate p = 400*(ln(gdpdef) - ln(L.gdpdef)) 
label variable p "Inflation rate"

In [ ]:
*********************************** Specifying the model to dsgenl **************************************************
dsgenl (1 = {beta}*(x/F.x)*(1/g)*(r/F.p))(1/{phi}+(p-1) = {phi}*x+{beta}*(F.p-1))({beta}*r = p^(1/{beta})*u) ///
(ln(F.u) = {rhou}*ln(u))(ln(F.g) = {rhog}*ln(g)), exostate(u g) observed(p r) unobserved(x)

In [ ]:
************* Parameter estimation and interpretation of nonlinear DSGEs ******************************************
dsgenl (1 = {beta}*(x/F.x)*(1/g)*(r/F.p))(1/{phi} + (p-1) = {phi}*x + {beta}*(F.p-1))({beta}*r = p^(1/{beta})*u) ///
 (ln(F.u) = {rhou}*ln(u))(ln(F.g) = {rhog}*ln(g)), exostate(u g) observed(p r) unobserved(x) nolog

In [ ]:
******************************* Parameter estimation and interpretation of nonlinear DSGEs **************************
nlcom 1/_b[beta]

**<span style="color:DarkGreen;font-size:150%">  An example: A linear DSGE model </span>**

In [ ]:
******************************* Specifying the model to dsge ********************************************************
dsge (p = {beta}*F.p + {kappa}*x)(x = F.x -(r - F.p - g), unobserved)(r = (1/{beta})*p + u)(F.u = {rhou}*u, state) ///
(F.g = {rhog}*g, state)

In [ ]:
******************************* Parameter estimation and interpretation of nonlinear DSGEs **************************
nlcom 1/_b[beta]

In [ ]:
****************************** Policy and transition matrices *******************************************************
estat policy
estat transition

****************************** Impulse responses ********************************************************************
irf set nkirf.irf
irf create model1
irf graph irf, impulse(u) response(x p r u) byopts(yrescale)

In [ ]:
********************************** Forecasts ************************************************************************ 
estimates store dsge_est // store the dsge estimation results
tsappend, add(12) // extend the dataset by 3 years, or 12 quarters
forecast create dsgemodel // initialize a new forecasting model, which we name dsgemodel
forecast estimates dsge_est // add the estimates from dsge to the forecasting model
forecast solve, prefix(d1_) begin(tq(2017q1)) // produce dynamic forecasts beginning in the first quarter of 2017
tsline d1_p if tin(2010q1, 2021q1), tline(2017q1) // We can graph the forecast for inflation d1 p using tsline
forecast solve, prefix(d2_) begin(tq(2014q1)) // compare the forecast for 2014–2016 
tsline p d2_p if tin(2010q1, 2021q1), tline(2014q1) // We plot both the observed inflation and the forecast

**<span style="color:DarkGreen;font-size:200%"> III.2 New Keynesian model </span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$$p_t = \beta E_t(p_{t+1}) + \kappa x_t~~~~~~~~~~~~~~~~~~~~~$$  

$$x_t = E_t(x_{t+1}) − {r_t − E_t(p_{t+1}) − g_t}$$  

$$r_t = \psi p_t + u_t~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$$  

$$u_{t+1} = \rho_u u_t + \epsilon_{t+1}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$$  

$$g_{t+1} = \rho_g g_t + \xi_{t+1}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$$

<span style="color:DarkGreen;font-size:150%"> Application in Stata </span> 

In [ ]:
******************************************** Parameter estimation ***************************************************
use https://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
constraint 1 _b[beta]=0.96 //We constrain β to be 0.96, a common value in the literature
dsge (p = {beta}*F.p + {kappa}*x)(x = F.x - (r - F.p - g), unobserved)(r = {psi}*p + u) ///
(F.u = {rhou}*u, state)(F.g = {rhog}*g, state), from(psi=1.5) constraint(1) 

In [ ]:
************************************ Policy and transition matrices *************************************************
estat policy //     response of a control variable to a one-unit increase in a state variable.
estat transition // elements of the state transition matrix 

In [ ]:
*********************************** One-step-ahead predictions ******************************************************
predict dep* // we obtain onestep-ahead predictions for each of the two observed control variables in the model
tsline p dep1, legend(col(1))

In [ ]:
********************************** Estimating an unobserved state ***************************************************
predict state1, state // observed control variables are driven by two unobserved state variables that we can predict
tsline state1

**<span style="color:DarkGreen;font-size:200%"> III.3 New Classical model </span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$\frac{1}{C_t} = \beta E_t \{\frac{1}{Cp_{t+1}}(1+R_{t+1} - \delta)\}~~~~~~~~~~~~~~~~$  

$H^\eta_t = \frac{W_t}{C_t}$  

$Y_t = C_t + X_t + G_t$  

$Y_t = K_t^\alpha(Z_tH_t)^{1−\alpha}$  

$W_t = (1 − \alpha)\frac{Y_t}{H_t}$  

$Rt = \alpha\frac{Y_t}{K_t}$  

$K_{t+1} = (1 − \delta)K_t + X_t$

<span style="color:peru">The linearized form of the model is</span>  


$c_t = E_t(c_{t+1}) − (1 − \beta + \beta\delta)E_t(r_{t+1})$  

$\eta h_t = w_t − c_t$  

$\phi_1x_t = y_t − \phi_2c_t − g_t$  

$y_t = (1 − \alpha)(z_t + h_t) + \alpha k_t$  

$w_t = y_t − h_t$  

$r_t = y_t − k_t$  

$k_{t+1} = \delta x_t + (1 − \delta)k_t$  

$z_{t+1} = \rho_zz_t + \epsilon_{t+1}$  

$g_{t+1} = \rho_gg_t + \xi_{t+1}$  


In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

dsge (c = F.c - (1-{beta}+{beta}*{delta})*F.r, unobserved)({eta}*h = w - c, unobserved) ///
    ({phi1}*x = y - {phi2}*c - g, unobserved)(y = (1-{alpha})*(z+h) + {alpha}*k)(w = y - h, unobserved) ///
    (r = y - k, unobserved)(F.k = {delta}*x+ (1-{delta})*k, state noshock)(F.z = {rhoz}*z, state) ///
    (F.g = {rhog}*g, state), from(beta=0.96 eta=1 alpha=0.3 delta=0.025 phi1=0.2 phi2=0.6 rhoz=0.8 rhog=0.3) ///
    solve noidencheck

In [ ]:
************************************ Policy and transition matrices *************************************************
estat transition

In [ ]:
**************************************** Impulse responses **********************************************************
irf set rbcirf
irf create persistent

irf graph irf, irf(persistent) impulse(z) response(y c x h w z) noci byopts(yrescale)

In [ ]:
********************************** Sensitivity analysis *************************************************************
/* we set the persistence of z to a smaller value of 0.6 to evaluate the role of persistence in z on the persistence 
 of other model variables */
dsge (c = F.c - F.r, unobserved)({eta}*h = w - c, unobserved)({phi1}*x = y - {phi2}*c - g, unobserved) ///
(y = (1-{alpha})*(z+h) + {alpha}*k)(w = y - h, unobserved)(r = y - k, unobserved) ///
(F.k = {delta}*x+ (1-{delta})*k, state noshock)(F.z = {rhoz}*z, state) ///
(F.g = {rhog}*g, state), from(eta=1 alpha=.3 delta=0.025 phi1=0.2 phi2=0.6 rhoz=0.6 rhog=0.3) solve noidencheck

In [ ]:
irf create transitory, replace

In [ ]:
irf graph irf, irf(transitory) impulse(z) response(y c x h w z) noci byopts(yrescale)

In [ ]:
irf ograph (persistent z c irf) (transitory z c irf) // This way we can view the differences across calibrations

**<span style="color:DarkGreen;font-size:200%"> III.4 Financial frictions model </span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$\pi_t = \beta E_t(\pi_{t+1}) + \kappa r_t$  

$x_t = E_tx_{t+1} − (i_t - E_t\pi_{t+1} - g_t)$  

$r_t = \psi\pi_t + u_t$  

$i_t = \chi r_t + e_t$  

$g_{t+1} = \rho_gg_t + \xi_{t+1}$  

$u_{t+1} = \rho_u u_t + \epsilon_{t+1}$  

$e_{t+1} = \rho_ee_t + \eta_{t+1}$  



<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear
constraint 1 _b[beta]=0.96

dsge (p = {beta}*F.p + {kappa}*x)(x = F.x - (i - F.p - g), unobserved)(i = {chi}*r + e)(r = {psi}*p + u) ///
(F.e = {rhoe}*e, state)(F.u = {rhou}*u, state)(F.g = {rhoz}*g, state), from(psi=2 chi=0.8) constraint(1)

In [ ]:
***************************** Policy and transition matrices ********************************************************
estat policy

In [ ]:
************************************** Impulse responses ************************************************************
irf set finirf
irf create param1
irf graph irf, irf(param1) impulse(e) response(e x p i r) byopts(yrescale)

**<span style="color:DarkGreen;font-size:200%"> III.5 Nonlinear New Keynesian model </span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$1 = \beta E_t(\frac{X_t}{X_{t+1}}\frac{1}{Z_t}\frac{R_t}{\Pi_{t+1}})$  

$(\theta -1)+\phi(\Pi_t-1)\Pi_t = \theta X_t + \phi\beta E_t\{(\Pi_{t+1} − 1)\Pi_{t+1}\}$  

$\beta R_t = \Pi_t^{\psi} M_t$  

$ln(M_{t+1}) = \rho_m ln(M_t) + u_{t+1}$  

$ln(Z_{t+1}) = \rho_z ln(Z_t) + e_{t+1}$  



<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

constraint 1 _b[theta]=5
constraint 2 _b[beta]=0.96

In [ ]:
dsgenl (1 = {beta}*(x/F.x)*(1/z)*(r/F.p))({theta}-1 + {phi}*(p -1)*p = {theta}*x + {phi}*{beta}*(F.p-1)*F.p) ///
(({beta})*r = (p)^({psi=2})*m)(ln(F.m) = {rhom}*ln(m))(ln(F.z) = {rhoz}*ln(z)), exostate(z m) unobserved(x) ///
observed(p r) constraint(1 2) nolog

In [ ]:
nlcom (_b[theta] - 1)/_b[phi] // k = (O-1)O

In [ ]:
*********************************** Policy and transition matrices **************************************************
estat policy
estat transition

In [ ]:
********************************** Impulse responses ****************************************************************
irf set nlex1irf, replace
irf create irf1, replace

irf graph irf, impulse(m z) response(x p r) byopts(yrescale)

In [ ]:
************************************ A change in constraints ********************************************************
quietly dsgenl
estimates store theta5

In [ ]:
constraint 3 _b[theta]=2
dsgenl (1 = {beta}*(x/F.x)*(1/z)*(r/F.p))({theta}-1 + {phi}*(p -1)*p = {theta}*x + {phi}*{beta}*(F.p-1)*F.p) ///
(({beta})*r = (p)^({psi=2})*m)(ln(F.m) = {rhom}*ln(m))(ln(F.z) = {rhoz}*ln(z)), exostate(z m) unobserved(x) ///
observed(p r) constraint(2 3) nolog

estimates store theta2

In [ ]:
estimates table theta5 theta2, b(%9.4f) stats(ll) // reported log-likelihood values are identical

In [ ]:
estimates restore theta5 // We compare k = (theta-1)theta accros two parametrizations
nlcom (_b[theta]-1) / _b[phi]

estimates restore theta2
nlcom (_b[theta]-1) / _b[phi]

In [ ]:
******** The equivalence of the two parameterizations can also be seen through the policy matrix *****************
estat policy // for theta = 2

In [ ]:
estimates restore theta5 
estat policy // for theta = 5

**<span style="color:DarkGreen;font-size:200%"> III.6 Nonlinear New Classical model </span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$\frac{1}{C_t} = \beta E_t \{\frac{1}{Cp_{t+1}}(1+R_{t+1} - \delta)\}~~~~~~~~~~~~~~~~$  

$\chi H_t = \frac{W_t}{C_t}$  

$Y_t = C_t + I_t$  

$Y_t = Z_tK_t^\alpha H_t^{1−\alpha}$  

$R_t = \alpha\frac{Y_t}{K_t}$  

$W_t = (1 − \alpha)\frac{Y_t}{H_t}$  

$K_{t+1} = I_t +(1 − \delta)K_t$  

$ln(Z_{t+1}) = \rho ln(Z_t) + e_{t+1}$ 

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
constraint 1 _b[alpha] = 0.33 // calibrated parameters which follows King and Rebelo (1999) that i allready download
constraint 2 _b[beta] = 0.99
constraint 3 _b[delta] = 0.025
constraint 4 _b[chi] = 2

In [ ]:
dsgenl (1/c = {beta}*(1/F.c)*(1+r-{delta}))({chi}*h = w/c)(y = c + i)(y = z*k^{alpha}*h^(1-{alpha})) ///
 (r = {alpha}*y/k)(w = (1-{alpha})*y/h)(F.k = i + (1-{delta})*k)(ln(F.z) = {rho}*ln(z)), observed(y) ///
 unobserved(c i r w h) exostate(z) endostate(k) constraint(1/4)

In [ ]:
**** Steady state: in the absence of shocks, variables converge to a steady-state position **************************
estat steady

In [ ]:
********** Model-implied covariances: state-space matrices generate predictions for variances and covariances
estat covariance

In [ ]:
****** relative volatility of a model variable = sd of variable vs with the sd of a reference variable *************
estimates store dsgenl
quietly estat covariance, post // post option to make the results stored in e(), where nlcom can access them

nlcom sqrt(_b[i:var(i)] / _b[y:var(y)]) // volatility of investment relative to output

In [ ]:
****************************  Policy and transition matrices  *******************************************************
estimates restore dsgenl
estat policy // sd of capital is constant over time because of constraint
estat transition // A 1% increase in productivity raises the future capital stock by 0.15%

In [ ]:
******************************* Impulse responses ******************************************************************
irf set rbcirf, replace
irf create est, step(20) replace
irf graph irf, impulse(z) response(y c i h w z) byopts(yrescale)

In [ ]:
***************************************** Sensitivity analysis ******************************************************
dsgenl // replay the current estimates.
matrix b = e(b) // store the current parameter vector in a Stata matrix
matrix b[1,5] = 0.6 // look at the response of the model to a shock when productivity is more persistent: rho = 0.6

In [ ]:
// from() to passed the parameter vector AND solve to have only parameters for policy and impule response
dsgenl (1/c = {beta}*(1/F.c)*(1+r-{delta}))({chi}/(1-h) = w/c)(y = c + i)(y = z*k^{alpha}*h^(1-{alpha})) /// 
 (w = (1-{alpha})*y/h)(r = {alpha}*y/k)(F.k = i + (1-{delta})*k)(ln(F.z) = {rho}*ln(z)), observed(y) /// 
 unobserved(c i r w h) exostate(z) endostate(k) from(b) solve noidencheck

In [ ]:
irf create alt, step(20) replace // We graph the response of y to a z impulse across the two models
irf ograph (est z y irf) (alt z y irf)

In [ ]:
foreach v in c h i w y z {
    irf ograph (est z ‘v’ irf) (alt z ‘v’ irf), name(‘v’) nodraw
}

graph combine c h i w y z    

**<span style="color:DarkGreen;font-size:200%"> III.7 Stochastic growth model </span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$1 = \beta E_t \{\frac{C_t}{Cp_{t+1}}(1+R_{t+1} - \delta)\}~~~~~~~~~~~~~~~~$  

$\chi H_t = \frac{W_t}{C_t}$  

$Y_t = Z_tK_t^\alpha$  

$R_t = \alpha Z_tK_t^{\alpha -1}$  

$K_{t+1} = Y_t -C_t + (1 − \delta)K_t$  

$ln(Z_{t+1}) = \rho ln(Z_t) + e_{t+1}$ 

<span style="color:peru"> The variables $Y_t, R_t, C_t, K_t$ and $Z_t$ are repectivelly **output, interest rate, concumption, capital** and **productivity**.</span>

<span style="color:DarkGreen;font-size:150%">Specifying the model to Stata</span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
matrix param_mat = (0.33, 0.96, 0.025, 0.9, 1) // we set up a parameter matrix containing the values that we will use
matrix colnames param_mat = alpha beta delta rho /sd(e.z)
matrix list param_mat //values are similar to those in King and Rebelo (1999) and Schmitt-Grohe and Uribe (2004)

In [ ]:
/* z is subject to shocks; it is an exogenous state variable. from() option specifies the starting values 
   for the parameters. The solve option declares that we wish to solve the model at the values in from() */
dsgenl (1 = {beta}*(F.c/c)^(-1)*(1+F.r-{delta})) (r = {alpha}*z*k^({alpha}-1)) (y = z*k^{alpha}) /// 
 (F.k = y - c + (1-{delta})*k) (ln(F.z) = {rho}*ln(z)) , observed(y) unobserved(c r) exostate(z) endostate(k) /// 
 from(param_mat) solve noidencheck

In [ ]:
********************************* After solving: steady state, policy and transition matrices ***********************
/* They display the percentage change in a control variable that results from a one-percent change in a 
state variable because by default, dsgenl uses a log-linear approximation */
estat steady, compact // compact to make state variables as columns. 

In [ ]:
********************* Linear and log-linear (default) approximations ************************************************
dsgenl (1 = {beta}*(F.c/c)^(-1)*(1+F.r-{delta})) (r = {alpha}*z*k^({alpha}-1)) (y = z*k^{alpha}) ///
 (F.k = y - c + (1-{delta})*k) (ln(F.z) = {rho}*ln(z)), observed(y) unobserved(c r) exostate(z) endostate(k) ///
 from(param_mat) solve noidencheck linearapprox // option linearapprox causes dsgenl to take a linear

In [ ]:
********************************** impact that the linearapprox option has ******************************************
estat steady, compact // No changes
estat policy, compact //one-unit increase in the capital stock leads output to rise by 0.067 units
estat transition, compact //one-unit increase in productivity Zt leads to a 1.54-unit increase in the capital stock 

**<span style="color:DarkGreen;font-size:200%"> III.8 Specifying a shock on a control variable</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$c_t = (1-h)w_t + hE_tc_{t+1} + \epsilon_t$  

$n_t = w_t - \gamma c_t$  

$w_{t+1} = \rho w_t + \xi_{t+1}$  

<span style="color:peru"> The variables $c_t, w_t$ and $n_t$ are repectivelly **concumption, wage** and **hours worked**. One cannot solve the model for the state-space form because the shock $\epsilon_t$ is added to the control equation $c_t$. We define: $z_t = \epsilon_t$ </span>

$c_t = (1-h)w_t + hE_tc_{t+1} + z_t$  

$n_t = w_t - \gamma c_t$  

$w_{t+1} = \rho w_t + \xi_{t+1}$  

$z_{t+1} = \epsilon_{t+1}$

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (c = (1-{h})*(w) + {h}*F.c + z)(n = w - {gamma}*c)(F.w = {rho}*w, state)(F.z = , state)

**<span style="color:DarkGreen;font-size:200%"> III.9 Including a lag of a control variable</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$p_t = \beta E_tp_{t+1} + \kappa y_t$  

$y_t = E_ty_{t+1} - (r_t - E_tp_{t+1}-\rho_zz_t)$  

$r_t = \rho_r r_{t-1} + (1 - \rho_r)(\frac{1}{\beta}p_t + u_t)$  

$z_{t+1} =  \rho_zz_t + \epsilon_{t+1}$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$

<span style="color:peru"> The variables $p_t, y_t$ and $r_t$ are repectivelly **inflation, output** and **interest rate**. One cannot solve the model the lag of a control variable does not fit
into the structure required. We define: $Lr_t = r_{t-1}$ </span>

$p_t = \beta E_tp_{t+1} + \kappa y_t$  

$y_t = E_ty_{t+1} - (r_t - E_tp_{t+1}-\rho_zz_t$  

$r_t = \rho_r Lr_t + (1 - \rho_r)(\frac{1}{\beta}p_t + u_t)$  

$Lr_{t+1} = r_t$  

$z_{t+1} =  \rho_zz_t + \epsilon_{t+1}$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - f.p - {rhoz}*z), unobserved) /// 
 (r = {rhor}*lr + (1-{rhor})*((1/{beta})*p + u))(F.lr = r, state noshock)(F.u = {rhou}*u, state)(F.z = {rhoz}*z, state)

**<span style="color:DarkGreen;font-size:200%"> III.10 Including a lag of a state variable</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$p_t = \beta E_t(p_{t+1}) + \kappa y_t$  

$y_t = E_t(y_{t+1}) - (r_t - E_t(p_{t+1}) - z_t)$  

$r_t = \frac{1}{\beta}p_t + u_t$  

$z_{t+1} =  \rho_{z1}z_t + \rho_{z2}z_{t-1} + \epsilon_{t+1}$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$

<span style="color:peru"> We define: $Lz_t = z_{t-1}$</span> 

$p_t = \beta E_t(p_{t+1}) + \kappa y_t$  

$y_t = E_t(y_{t+1}) - (r_t - E_t(p_{t+1}) - z_t)$  

$r_t = \frac{1}{\beta}p_t + u_t$  

$z_{t+1} =  \rho_{z1}z_t + \rho_{z2}Lz_t + \epsilon_{t+1}$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$  

$Lz_{t+1} = z_t$

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - F.p - z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz1}*z + {rhoz2}*Lz, state)(F.Lz = z, state noshock)

**<span style="color:DarkGreen;font-size:200%"> III.11 Including an expectation dated by more than one period ahead</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$c_t = (1-h)w_t + hE_tc_{t+2} + r_t$  

$n_t = w_t - \gamma c_t$  

$w_{t+1} = \rho_w w_t + \xi_{t+1}$  

$r_{t+1} = \rho_r r_t + \epsilon_{t+1}$

<span style="color:peru"> We define: $Fc_t = c_{t+1}$</span> 

$c_t = (1-h)w_t + hE_t(Fc_{t+1}) + r_t$  

$n_t = w_t - \gamma c_t$  

$Fc_t = c_{t+1}$  

$w_{t+1} = \rho_w w_t + \xi_{t+1}$  

$r_{t+1} = \rho_r r_t + \epsilon_{t+1}$

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (c = (1-{h})*(w) + {h}*F.fc + r)(n = w - {gamma}*c)(fc = F.c, unobserved)(F.w = {rho_w}*w, state) /// 
 (F.r = {rho_r}*r, state)

**<span style="color:DarkGreen;font-size:200%"> III.12 Including a second-order lag of a control</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$n_t = b_1n_{t-2} + w_t - \gamma c_t$  

$c_t = (1-h)w_t + hE_tc_{t+1} + r_t$  

$w_{t+1} = \rho w_t + \xi_{t+1}$  

$r_{t+1} = \epsilon_{t+1}$

<span style="color:peru"> We define: $Ln_t = n_{t-1}, ~~L2n_t = Ln_{t-1}$</span> 

$n_t = b_1L2n_t + w_t - \gamma c_t$  

$c_t = (1-h)w_t + hE_tc_{t+1} + r_t$  

$w_{t+1} = \rho w_t + \xi_{t+1}$  

$r_{t+1} = \epsilon_{t+1}$  

$Ln_{t+1} = n_t$  

$L2n_{t+1} = Ln_t$

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (n = {b1}*L2n + w - {gamma}*c)(c = (1-{h})*w + {h}*F.c + r)(F.w = {rho}*w, state)(F.r = , state) ///
 (F.L2n = Ln, state noshock)(F.Ln = n, state noshock)

**<span style="color:DarkGreen;font-size:200%"> III.13 Including an observed exogenous variable</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$x_t = E_t(x_{t+1}) − (r_t - E_t(p_{t+1}) - g_t)$  

$r_t = \frac{1}{\beta}p_t + u_t$  

$p_t = \beta E_t(p_{t+1}) + \kappa x_t + \psi e_t$  

$u_{t+1} = \rho_u u_t + \epsilon_{t+1}$  

$g_{t+1} = \rho_gg_t + \xi_{t+1}$  

<span style="color:peru"> We define: $e_t = es_t$  


$x_t = E_t(x_{t+1}) − (r_t - E_t(p_{t+1}) - g_t)$  

$r_t = \frac{1}{\beta}p_t + u_t$  

$p_t = \beta E_t(p_{t+1}) + \kappa x_t + \psi es_t$  

$e_t = es_t$  

$u_{t+1} = \rho_u u_t + \epsilon_{t+1}$  

$g_{t+1} = \rho_gg_t + \xi_{t+1}$  
    
$es_{t+1} = \rho_ees_t + \eta_{t+1}$

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (x = F.x - (r - F.p - g), unobserved)(r = 1/{beta}*p + u)(p = {beta}*F.p + {kappa}*x + {psi}*es)(e = es) /// 
 (F.u = {rho_u}*u, state)(F.g = {rho_g}*g, state)(F.es = {rho_e}*es, state)

**<span style="color:DarkGreen;font-size:200%"> III.14 Correlated state variables</span>**

<span style="color:DarkGreen;font-size:150%"> The model </span> 

$y_t = E_t(y_{t+1}) + \alpha p_t + g_t)$  

$p_t = z_t$  

$g_{t+1} = \rho_gg_t + \rho_{gz} z_t + \xi_{t+1}$  

$z_{t+1} = \rho_z z_t + \epsilon_{t+1}$  

<span style="color:DarkGreen;font-size:150%">Parameter estimation </span> 

In [ ]:
dsge (y = F.y + {alpha}*p + g)(p = z)(F.g = {rho_g}*g + {rho_gz}*z, state)(F.z = {rho_z}*z, state)

**<span style="color:DarkGreen;font-size:200%"> III.15 Stability conditions</span>**

<span style="color:DarkGreen;font-size:150%"> Finding saddle-path stable initial values </span> 

$p_t = \frac{1}{\gamma} E_t(p_{t+1}) + \kappa y_t$  

$y_t = E_t(y_{t+1}) - (r_t - E_t(p_{t+1}) - z_t)$  

$r_t = \gamma p_t + u_t$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$ 

$z_{t+1} =  \rho_{z1}z_t + \rho_{z2}Lz_t + \epsilon_{t+1}$  

$Lz_{t+1} = z_t$

In [ ]:
*********************** We try to fit this model using default starting values below ********************************
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (p = (1/{gamma})*F.p + {kappa}*y)(y = F.y - (r - F.p - z), unobserved)(r = {gamma}*p + u) /// 
 (F.u = {rho_u}*u, state)(F.z = {rho_z1}*z + {rho_z2}*Lz, state)(F.Lz = z, state noshock)

In [ ]:
*************  specify options solve and noidencheck to get default initial values **********************************
dsge (p = (1/{gamma})*F.p + {kappa}*y) (y = F.y - (r - F.p - z), unobserved) (r = {gamma}*p + u) /// 
 (F.u = {rho_u}*u, state) (F.z = {rho_z1}*z + {rho_z2}*Lz, state) (F.Lz = z, state noshock), solve noidencheck

In [ ]:
**************** We use estat stable to look at the eigenvalues implied by the initial values.***********************
estat stable

In [ ]:
******** gamma must be greater than one, we use from() to fixe a right initial value of gamma *********************
dsge (p = (1/{gamma})*F.p + {kappa}*y)(y = F.y - (r - F.p - z), unobserved)(r = {gamma}*p + u) /// 
 (F.u = {rho_u}*u, state)(F.z = {rho_z1}*z + {rho_z2}*Lz, state) /// 
 (F.Lz = z, state noshock), solve noidencheck from(gamma=1.2)

In [ ]:
*********************************** No warning messages above, so check for stability *******************************
estat stable

**<span style="color:DarkGreen;font-size:200%"> III.16 Identification </span>**

<span style="color:DarkGreen;font-size:150%"> A model with two unidentified parameters </span> 

$p_t = \beta E_t(p_{t+1}) + \kappa y_t$  

$y_t = E_t(y_{t+1}) - \gamma(r_t - E_t(p_{t+1}) - \rho z_t)$  

$\beta r_t = p_t + \beta u_t$  

$z_{t+1} =  \rho z_t + \epsilon_{t+1}$  

$u_{t+1} = \delta u_t + \xi_{t+1}$  


<span style="color:peru">Consider the above model of inflation $p_t$, output growth $y_t$, and the interest rate $r_t$.

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y -{gamma}*(r - F.p - {rhoz}*z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state)

In [ ]:
constraint 2 _b[gamma]=1 // because of identification failure: {kappa} {gamma}

In [ ]:
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y -{gamma}*(r - F.p - {rhoz}*z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state), constraint(2)

In [ ]:
************************ Suppressing the identification check *******************************************************
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y -{gamma}*(r - F.p - {rhoz}*z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state), noidencheck iterate(50) // iterate() to not iterate indefinitely

**<span style="color:DarkGreen;font-size:200%"> III.17 Convergence problems </span>**

<span style="color:DarkGreen;font-size:150%"> initial values of $\beta = 0.5, \kappa = 0.2, \rho_u = 0.7, \rho_{z1} = 0.7$, and $\rho_{z2} = 0.2$. Inside by typing {param = value} </span> 

$p_t = \beta E_t(p_{t+1}) + \kappa y_t$  

$y_t = E_t(y_{t+1}) - (r_t - E_t(p_{t+1}) - z_t)$  

$r_t = \frac{1}{\beta}p_t + u_t$  

$z_{t+1} =  \rho_{z1}z_t + \rho_{z2}Lz_t + \epsilon_{t+1}$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$  

$Lz_{t+1} = z_t$

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (p = {beta=.5}*F.p + {kappa=.2}*y)(y = F.y - (r - F.p - z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rho_u=.7}*u, state)(F.z = {rho_z1=.7}*z + {rho_z2=.2}*Lz, state)(F.Lz = z, state noshock)

<span style="color:DarkGreen;font-size:150%">Specifying starting values using from()</span> 

In [ ]:
******************************* Specifying starting values using from() *********************************************
matrix ivalues = (.5, .2, .7, .7, .2, 2.3, .7)
matrix list ivalues

In [ ]:
************ Specifying from(, copy) implies first element in the vector = initial value for the first etc..*********
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - F.p - z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rho_u}*u, state)(F.z = {rho_z1}*z + {rho_z2}*Lz, state)(F.Lz = z, state noshock), from(ivalues, copy)

<span style="color:DarkGreen;font-size:150%">Starting with a more constrained model to obtain convergence</span> 

In [ ]:
************ Starting with a more constrained model to obtain convergence *******************************************
constraint define 1 _b[beta] = 0.5
constraint define 2 _b[kappa] = 0.1
constraint define 3 _b[rho_z2] = 0.01

dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - F.p - z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rho_u}*u, state)(F.z = {rho_z1}*z + {rho_z2}*Lz, state)(F.Lz = z, state noshock), constraints(1 2 3)

In [ ]:
matrix b = e(b) // We use this parameters as initial values 
matrix list b

In [ ]:
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - F.p - z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rho_u}*u, state)(F.z = {rho_z1}*z + {rho_z2}*Lz, state)(F.Lz = z, state noshock), from(b, copy)

<span style="color:DarkGreen;font-size:150%"> We can specify *technique(bfgs 200 nr)*</span> 

**<span style="color:DarkGreen;font-size:200%"> III.18 Wald tests vary with nonlinear transforms </span>**

<span style="color:DarkGreen;font-size:150%"> The Model </span> 

$p_t = \beta E_t(p_{t+1}) + \kappa y_t$  

$y_t = E_t(y_{t+1}) - (r_t - E_t(p_{t+1}) - z_t)$  

$r_t = \frac{1}{\beta}p_t + u_t$  

$z_{t+1} =  \rho z_t + \epsilon_{t+1}$  

$u_{t+1} = \rho_uu_t + \xi_{t+1}$  

In [ ]:
use http://www.stata-press.com/data/r16/usmacro2, clear

In [ ]:
dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - F.p - {rhoz}*z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state)

<span style="color:DarkGreen;font-size:150%"> Wald tests vary with nonlinear transforms</span> 

In [ ]:
** We test whether the coefficient on inflation parameter (1/beta) is 1.5, a common benchmark value in the literature
testnl 1/_b[beta] = 1.5 // Test nonlinear hypotheses after estimation

In [ ]:
******************* Wald tests are not invariant to nonlinear transformation ****************************************
test _b[beta] =2/3 // Test linear hypotheses after estimation

<span style="color:DarkGreen;font-size:150%"> LR (Likelihood-ratio) tests do not vary with nonlinear transforms</span> 

In [ ]:
dsge // to active current estimation

In [ ]:
estimates store unconstrained

In [ ]:
constraint 1 _b[beta] = 2/3

dsge (p = {beta}*F.p + {kappa}*y)(y = F.y - (r - F.p - {rhoz}*z), unobserved)(r = (1/{beta})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state), constraint(1)

estimates store constrained

In [ ]:
lrtest unconstrained constrained // Not significant so model are the same

In [ ]:
********** We now illustrate an LR of the null hypothesis that 1=β = 1.5 produces the same value *******************

* We write the model in terms of {gamma}=1/{beta} 
quietly dsge (p = 1/{gamma}*F.p + {kappa}*y)(y = F.y - (r - F.p - {rhoz}*z), unobserved)(r = ({gamma})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state), from(gamma=2 kappa=0.15 rhou=0.75 rhoz=0.95)
estimates store unconstrained2

constraint 2 _b[gamma] = 1.5 // Same to beta = 2/3
quietly dsge (p = 1/{gamma}*F.p + {kappa}*y)(y = F.y - (r - F.p - {rhoz}*z), unobserved)(r = ({gamma})*p + u) /// 
 (F.u = {rhou}*u, state)(F.z = {rhoz}*z, state), constraint(2)
estimates store constrained2

lrtest unconstrained2 constrained2

# **<span style="color:crimson;font-size:150%"> IV. Extended Regression Models</span>**

<span style="color:peru"> The main commands used here are:</span>  
    
<span style="color:peru">1. eregress</span>  
<span style="color:peru">2. eintreg</span>  
<span style="color:peru">3. eprobit</span>  
<span style="color:peru">4. eoprobit</span>    

<span style="color:peru">And for panel or group models:</span> 
<span style="color:peru">1. xteregress</span>  
<span style="color:peru">2. xteintreg</span>  
<span style="color:peru">3. xteprobit</span>  
<span style="color:peru">4. xteoprobit</span>

**<span style="color:DarkGreen;font-size:150%"> How to use margins in models without endogenous covariates </span>**

**<span style="color:peru;font-size:120%">The model fit is</span>**  

$y_i = \beta_0 + \beta_1x1_i + \beta_2x2_i + \beta_3x1_ix2_i + e_i.y$

In [ ]:
use http://www.stata-press.com/data/r16/ermexample, clear

In [ ]:
eregress y x1 x2 c.x1#c.x2

In [ ]:
margins, at(x1=generate(x1)) at(x1=generate(x1+1)) contrast(at(r) nowald) over(group)

**<span style="color:DarkGreen;font-size:150%"> The two ways to use margins with endogenous covariates </span>**

In [ ]:
eregress y x1 x2, endogenous(x1 = z1, nomain)

**<span style="color:DarkGreen;font-size:150%"> Using margins with nonlinear and random-effects models </span>**

In [ ]:
xteregress y x1 x2, endogenous(x1 = z1, nomain)

**<span style="color:DarkGreen;font-size:150%"> How to use margins with predict(base())</span>**

In [ ]:
eregress y x1 x2 c.x1#c.x2
margins, at(x1=generate(x1)) at(x1=generate(x1+1)) contrast(at(r))

* To produce the same counterfactual, we type
generate x1orig = x1
margins, at(x1=generate(x1)) at(x1=generate(x1+1)) contrast(at(r)) predict(base(x1=x1orig))

* If we wanted a comparison of x1+1 with x1+2, we would type
generate x1orig = x1
margins, at(x1=generate(x1+1)) at(x1=generate(x1+2)) contrast(at(r)) predict(base(x1=x1orig))

* If we requested counterfactuals that involved changing x1 and x2, we would type
generate x1orig = x1
generate x2orig = x2
margins, at(x1=generate(x1) x2=generate(x2)) at(x1=generate(x1+1) x2=generate(x2+1)) ///
 contrast(at(r)) predict(base(x1=x1orig x2=x2orig))

* If we requested counterfactuals that involved changing x1 and x2, we would type
generate x1orig = x1
generate x2orig = x2
margins, at(x1=generate(x1) x2=generate(x2)) at(x1=generate(x1+1) x2=generate(x2+1)) ///
 contrast(at(r)) predict(base(x1=x1orig x2=x2orig))

# **<span style="color:crimson;font-size:150%"> V. Finite Mixture Models</span>**

# **<span style="color:crimson;font-size:150%"> VI. Item Response Theory</span>**

# **<span style="color:crimson;font-size:150%"> VII. Lasso </span>**

# **<span style="color:crimson;font-size:150%"> VIII. Longitudinal-Data/Panel-Data </span>**

# **<span style="color:crimson;font-size:150%"> IX. Meta-Analysis </span>**

# **<span style="color:crimson;font-size:150%"> X. Multilevel Mixed-Effects </span>**

# **<span style="color:crimson;font-size:150%"> XI. Multiple-Imputation </span>**

# **<span style="color:crimson;font-size:150%"> XII. Multivariate Statistics </span>**

# **<span style="color:crimson;font-size:150%"> XIII. Power, Precision, and Sample-Size </span>**

# **<span style="color:crimson;font-size:150%"> XIV. Spatial Autoregressive Models </span>**

# **<span style="color:crimson;font-size:150%"> XV. Structural Equation Modeling </span>**

# **<span style="color:crimson;font-size:150%"> XVI. Survey Data </span>**

## <span style="color:gold;font-size:150%"> XVI.1 Introduction to survey data manual </span>   


The outline is prensented alphabetically

## <span style="color:gold;font-size:150%"> XVI.2 Introduction to survey commands </span>   


The outline is prensented by group of commands according to the functions their perform:  
- survey design tools (svyset, svydescribe)  
- survey data analysis tools (14 commands: **<span style="color:green">svy, svy estimation, svy tabulation oneway, svy tabulation twoway, svy postestimation, estat, svy boostrap, boostrap_options, svy brr, brr_options, svy jakknife, jakknife_options, svy sdr, sdr_options</span>**)  
- survey data concepts (**<span style="color:green">variance estimation, subpopulation estimation, calibration, direct standardization, postratification</span>**)   
- tools for programmers of new survey commands (**<span style="color:green">ml for svy, markout</span>**)

In [1]:
* One stage design survey data with: su1 = PSU
use http://www.stata-press.com/data/r16/stage5a, clear


In [2]:
* We define data as survey data
svyset su1 [pweight=pw], strata(strata) fpc(fpc1)


      pweight: pw
          VCE: linearized
  Single unit: missing
     Strata 1: strata
         SU 1: su1
        FPC 1: fpc1


For two levels servey design: first level is _country_ selected within each state (the strata), and the second level is the _schools_ selected hithin each country. **samwgt** is the weight of individual

In [3]:
* Multistage survey design
use http://www.stata-press.com/data/r16/multistage, clear


In [8]:
di _N

4071


In [5]:
svyset county [pw=sampwgt], strata(state) fpc(ncounties) || school, fpc(nschools)



      pweight: sampwgt
          VCE: linearized
  Single unit: missing
     Strata 1: state
         SU 1: county
        FPC 1: ncounties
     Strata 2: <one>
         SU 2: school
        FPC 2: nschools


In [6]:
* We use svydescribe varlist: to putout sampling design informations and missing values of varlist
svydescribe weight


Survey: Describing stage 1 sampling units

      pweight: sampwgt
          VCE: linearized
  Single unit: missing
     Strata 1: state
         SU 1: county
        FPC 1: ncounties
     Strata 2: <one>
         SU 2: school
        FPC 2: nschools

                             #Obs with  #Obs with     #Obs per included Unit
           #Units    #Units   complete  missing   ----------------------------
Stratum   included  omitted     data      data      min       mean      max   
--------  --------  --------  --------  --------  --------  --------  --------
       1         2         0        92         0        34      46.0        58
       2         2         0       112         0        51      56.0        61
       3         2         0        43         0        18      21.5        25
       4         2         0        37         0        14      18.5        23
       5         2         0        96         0        38      48.0        58
       6         2         0        76 

In [10]:
* Survey data analysis tools
*1. Estimation of population mean
svy: mean weight


(running mean on estimation sample)

Survey: Mean estimation

Number of strata =      50        Number of obs   =      4,071
Number of PSUs   =     100        Population size =  8,000,000
                                  Design df       =         50

--------------------------------------------------------------
             |             Linearized
             |       Mean   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
      weight |   160.2863   .7412512      158.7974    161.7751
--------------------------------------------------------------


In [11]:
*2. Regression to get association between weight and height
svy: regress weight height


(running regress on estimation sample)

Survey: Linear regression

Number of strata   =        50                  Number of obs     =      4,071
Number of PSUs     =       100                  Population size   =  8,000,000
                                                Design df         =         50
                                                F(   1,     50)   =     593.99
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2787

------------------------------------------------------------------------------
             |             Linearized
      weight |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      height |   .7163115   .0293908    24.37   0.000     .6572784    .7753447
       _cons |  -149.6183   12.57265   -11.90   0.000    -174.8712   -124.3654
-------------------------

In [15]:
* Cox’s proportional hazards model
use http://www.stata-press.com/data/r16/nhefs, clear


In [16]:
svyset psu2 [pw=swgt2], strata(strata2)


      pweight: swgt2
          VCE: linearized
  Single unit: missing
     Strata 1: strata2
         SU 1: psu2
        FPC 1: <zero>


In [18]:
stset age_lung_cancer [pw=swgt2], fail(lung_cancer) // declare data as survival time data


     failure event:  lung_cancer != 0 & lung_cancer < .
obs. time interval:  (0, age_lung_cancer]
 exit on or before:  failure
            weight:  [pweight=swgt2]

------------------------------------------------------------------------------
     14,407  total observations
      5,126  event time missing (age_lung_cancer>=.)            PROBABLE ERROR
------------------------------------------------------------------------------
      9,281  observations remaining, representing
         83  failures in single-record/single-failure data
    599,691  total analysis time at risk and under observation
                                                at risk from t =         0
                                     earliest observed entry t =         0
                                          last observed exit t =        97


In [19]:
* Running Cox regression in survival model
svy: stcox former_smoker smoker male urban1 rural

(running stcox on estimation sample)

Survey: Cox regression

Number of strata   =        35                Number of obs     =        9,149
Number of PSUs     =       105                Population size   =  151,327,827
                                              Design df         =           70
                                              F(   5,     66)   =        14.07
                                              Prob > F          =       0.0000

-------------------------------------------------------------------------------
              |             Linearized
           _t | Haz. Ratio   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
former_smoker |   2.788113   .6205102     4.61   0.000     1.788705    4.345923
       smoker |   7.849483   2.593249     6.24   0.000     4.061457    15.17051
         male |   1.187611   .3445315     0.59   0.555     .6658757    2.118142
       urban1 |   .8035

In [20]:
* Two-way tables for survey data
use http://www.stata-press.com/data/r16/nhanes2b, clear
svydescribe




Survey: Describing stage 1 sampling units

      pweight: finalwgt
          VCE: linearized
  Single unit: missing
     Strata 1: stratid
         SU 1: psuid
        FPC 1: <zero>

                                      #Obs per Unit
                              ----------------------------
Stratum    #Units     #Obs      min       mean      max   
--------  --------  --------  --------  --------  --------
       1         2       380       165     190.0       215
       2         2       185        67      92.5       118
       3         2       348       149     174.0       199
       4         2       460       229     230.0       231
       5         2       252       105     126.0       147
       6         2       298       131     149.0       167
       7         2       476       206     238.0       270
       8         2       338       158     169.0       180
       9         2       244       100     122.0       144
      10         2       262       119     131.0      

In [22]:
svy: tabulate race diabetes, row se ci format(%7.4f)

(running tabulate on estimation sample)

Number of strata   =        31                Number of obs     =       10,349
Number of PSUs     =        62                Population size   =  117,131,111
                                              Design df         =           31

-------------------------------------------------------------
1=white,  |
2=black,  |               diabetes, 1=yes, 0=no              
3=other   |               0                1            Total
----------+--------------------------------------------------
    White |          0.9680           0.0320           1.0000
          |        (0.0020)         (0.0020)                 
          | [0.9638,0.9718]  [0.0282,0.0362]                 
          | 
    Black |          0.9410           0.0590           1.0000
          |        (0.0061)         (0.0061)                 
          | [0.9271,0.9523]  [0.0477,0.0729]                 
          | 
    Other |          0.9797           0.0203           1.0000
 

In [31]:
* Comparing means
use http://www.stata-press.com/data/r16/highschool, clear // already svyset


In [38]:
svy: mean weight, over(sex)


(running mean on estimation sample)

Survey: Mean estimation

Number of strata =      50        Number of obs   =      4,071
Number of PSUs   =     100        Population size =  8,000,000
                                  Design df       =         50

         male: sex = male
       female: sex = female

--------------------------------------------------------------
             |             Linearized
        Over |       Mean   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
weight       |
        male |   175.4809   1.116802      173.2377    177.7241
      female |    146.204   .9004157      144.3955    148.0125
--------------------------------------------------------------


In [36]:
* test after mean
test 1.sex



Adjusted Wald test
1.sex not found


r(111);


In [33]:
* test after mean
test weight#1.sex - weight#2.sex = 30



Adjusted Wald test
variable sex not found


r(111);


In [37]:
* reports the survey design settings
estat svyset
estat effects
estat lceffects
estat size // report table of population and subpopulation after svy: mean; proportion; ratio; total
estat sd // standard deviation of subpopulation after svy: mean
estat strata // report number of singletons 
estat cv // report coefficient variation of each coefficient in current estimation
estat gof // report goodness of fit test after binary model



      pweight: sampwgt
          VCE: linearized
  Single unit: missing
     Strata 1: state
         SU 1: county
        FPC 1: ncounties
     Strata 2: <one>
         SU 2: school
        FPC 2: nschools


In [39]:
estat effects


         male: sex = male
       female: sex = female

----------------------------------------------------------
             |             Linearized
        Over |       Mean   Std. Err.       DEFF      DEFT
-------------+--------------------------------------------
weight       |
        male |   175.4809   1.116802     2.61016   1.61519
      female |    146.204   .9004157      1.7328   1.31603
----------------------------------------------------------
Note: Weights must represent population totals for deff to
      be correct when using an FPC; however, deft is
      invariant to the scale of weights.


In [40]:
estat lceffects weight#1.sex - weight#2.sex

variable sex not found


r(111);


Estimation of variance with four methods:   
1. BRR (Balanced Repeated Replication)  
2. Jakknife replication   
3. Boostrap
4. SDR (Successive Difference Replication)   


**Boostrap** and **SDR** work only with externly weight

In [43]:
* BRR and replicate-weight variables
use http://www.stata-press.com/data/r16/nhanes2, clear
svydescribe





Survey: Describing stage 1 sampling units

      pweight: finalwgt
          VCE: linearized
  Single unit: missing
     Strata 1: strata
         SU 1: psu
        FPC 1: <zero>

                                      #Obs per Unit
                              ----------------------------
Stratum    #Units     #Obs      min       mean      max   
--------  --------  --------  --------  --------  --------
       1         2       380       165     190.0       215
       2         2       185        67      92.5       118
       3         2       348       149     174.0       199
       4         2       460       229     230.0       231
       5         2       252       105     126.0       147
       6         2       298       131     149.0       167
       7         2       476       206     238.0       270
       8         2       338       158     169.0       180
       9         2       244       100     122.0       144
      10         2       262       119     131.0       14

In [44]:
* new dataset allready svyset with  brr replicate weight
use http://www.stata-press.com/data/r16/nhanes2brr, clear


In [45]:
svyset


      pweight: finalwgt
          VCE: brr
          MSE: off
    brrweight: brr_1 .. brr_32
  Single unit: missing
     Strata 1: <one>
         SU 1: <observations>
        FPC 1: <zero>


In [46]:
svy brr WtoH = (_b[weight]/_b[height]): total weight height

(running total on estimation sample)

BRR replications (32)
----+--- 1 ---+--- 2 ---+--- 3 ---+--- 4 ---+--- 5 
................................

BRR results                                   Number of obs     =       10,351
                                              Population size   =  117,157,513
                                              Replications      =           32
                                              Design df         =           31

      command:  total weight height
         WtoH:  _b[weight]/_b[height]

------------------------------------------------------------------------------
             |                 BRR
             |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        WtoH |   .4268116   .0008904   479.36   0.000     .4249957    .4286276
------------------------------------------------------------------------------


In [47]:
* Subpopulation estimation
use http://www.stata-press.com/data/r16/highschool, clear


In [48]:
describe sex
label list sex



              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
sex             byte    %9.0g      sex        1=male, 2=female

sex:
           1 male
           2 female


In [49]:
* Subpopulation specification
generate male = sex == 1 if !missing(sex)
svy, subpop(male): regress weight height




(running regress on estimation sample)

Survey: Linear regression

Number of strata   =        50                  Number of obs     =      4,071
Number of PSUs     =       100                  Population size   =  8,000,000
                                                Subpop. no. obs   =      1,938
                                                Subpop. size      =  3,848,021
                                                Design df         =         50
                                                F(   1,     50)   =     225.38
                                                Prob > F          =     0.0000
                                                R-squared         =     0.2347

------------------------------------------------------------------------------
             |             Linearized
      weight |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      height |   .76329

In [50]:
* Standardized rates
use http://www.stata-press.com/data/r16/stdize, clear


In [51]:
gen fpc = 1 // specify a census data: fpc = 100%
svyset, fpc(fpc)





      pweight: <none>
          VCE: linearized
  Single unit: missing
     Strata 1: <one>
         SU 1: <observations>
        FPC 1: fpc


In [52]:
svy: ratio (Bethnal: bgdeaths/bgliving)(Hanover: hsdeaths/hsliving)

(running ratio on estimation sample)

Survey: Ratio estimation

Number of strata =       1        Number of obs   =         21
Number of PSUs   =      21        Population size =         21
                                  Design df       =         20

      Bethnal: bgdeaths/bgliving
      Hanover: hsdeaths/hsliving

--------------------------------------------------------------
             |             Linearized
             |      Ratio   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
     Bethnal |   .0238095          0             .           .
     Hanover |   .0213384          0             .           .
--------------------------------------------------------------
Note: Zero standard errors because of 100% sampling rate
      detected for FPC in the first stage.


We standardize _age_ classes to compare the two ratios using _bgliving_ as population sizes

In [53]:
svy: ratio (Bethnal: bgdeaths/bgliving) (Hanover: hsdeaths/hsliving), stdize(age) stdweight(bgliving)

(running ratio on estimation sample)

Survey: Ratio estimation

Number of strata =       1        Number of obs   =         21
Number of PSUs   =      21        Population size =         21
N. of std strata =      21        Design df       =         20

      Bethnal: bgdeaths/bgliving
      Hanover: hsdeaths/hsliving

--------------------------------------------------------------
             |             Linearized
             |      Ratio   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
     Bethnal |   .0238095          0             .           .
     Hanover |   .0266409          0             .           .
--------------------------------------------------------------
Note: Zero standard errors because of 100% sampling rate
      detected for FPC in the first stage.


In [54]:
* Poststratified mean
use http://www.stata-press.com/data/r16/poststrata, clear


In [55]:
estat svyset



      pweight: <none>
          VCE: linearized
  Single unit: missing
     Strata 1: <one>
         SU 1: <observations>
        FPC 1: fpc


In [56]:
svyset, poststrata(type) postweight(postwgt) fpc(fpc)



      pweight: <none>
          VCE: linearized
   Poststrata: type
   Postweight: postwgt
  Single unit: missing
     Strata 1: <one>
         SU 1: <observations>
        FPC 1: fpc


In [57]:
svy: mean totexp


(running mean on estimation sample)

Survey: Mean estimation

Number of strata =       1        Number of obs   =         50
Number of PSUs   =      50        Population size =      1,300
N. of poststrata =       2        Design df       =         49

--------------------------------------------------------------
             |             Linearized
             |       Mean   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
      totexp |   40.11513   1.163498      37.77699    42.45327
--------------------------------------------------------------


In [58]:
* Whithout postratification
svyset, fpc(fpc)
svy: mean totexp




      pweight: <none>
          VCE: linearized
  Single unit: missing
     Strata 1: <one>
         SU 1: <observations>
        FPC 1: fpc

(running mean on estimation sample)

Survey: Mean estimation

Number of strata =       1        Number of obs   =         50
Number of PSUs   =      50        Population size =         50
                                  Design df       =         49

--------------------------------------------------------------
             |             Linearized
             |       Mean   Std. Err.     [95% Conf. Interval]
-------------+------------------------------------------------
      totexp |    39.7254   2.221747      35.26063    44.19017
--------------------------------------------------------------


In [ ]:
* Tools for programmers of new survey commands
* A voir

## <span style="color:gold;font-size:150%"> XVI.3 More options for bootstrap and Balanced Repeated Replication (BRR) variance estimation </span>   


_**mse, nodots, dots(#), \[bsn(#) ou hadamard(matrix), fay(#)\], saving(filename,...), verbose, noisily, trace, title(text), nodrop, reject(exp)**_

## <span style="color:gold;font-size:150%"> XVI.4 Calibration for survey data </span>   



# **<span style="color:crimson;font-size:150%"> XVII. Survival Analysis</span>**

# **<span style="color:crimson;font-size:150%"> XVIII. Time Series</span>**

# **<span style="color:crimson;font-size:150%"> XIX. Treatment Effects</span>**

# **<span style="color:crimson;font-size:150%"> END of chapters </span>**

In [ ]:
exit, clear

In [1]:
%show_gui